<a href="https://colab.research.google.com/github/SaiGanesh26/Gender-Classification-Age-Prediction/blob/master/AgeGender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Age Prediction & Gender Classification

###Imports

In [49]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.models import load_model
# from google.colab import  files
# from google.colab.patches import cv2_imshow

####Using OpenCV haar cascade classifiers for face detection 

In [50]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

####Detecting and drawing shapes across the face
* Face is detected across each frame if video input is given

In [29]:
def detect_face_image(img):
    label_dict={0:'Female',1:'Male'}
    color_dict={0:(0,255,0),1:(0,0,255)}
    
    face_coord = face_cascade.detectMultiScale(img,scaleFactor = 1.2,minNeighbors = 5)  # This gives us the coordinates of rectangle drawn across the face 
    for x,y,w,h in face_coord:
        image = img[y:y+h,x:x+h]
        image_gry = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        resize_img = cv2.resize(image_gry,(200,200))
        norm_img = resize_img/255.0
        reshaped=np.reshape(norm_img,(1,200,200,1))
        #reshaped = np.vstack([reshaped])

        try:  
            with tf.device('/device:GPU:5'):
                gender_res = gen_model.predict(reshaped)
        except RuntimeError as e:
            print(e)
        if(gender_res[0][0]<0.5):
            label=0
        else:
            label=1
     
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],thickness = 5)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, label_dict[label], (x, y-40),cv2.FONT_HERSHEY_SIMPLEX,5,(255,255,255),5)
    return img

In [31]:
# image = cv2.imread("IMG_2137.JPG")
# out_image = detect_face_image(image)
# plt.imshow(out_image)

####Video Input
* If existing video file is to be used then file path is given
* If live web cam is used just enter 0

In [51]:
video = cv2.VideoCapture('test3.mp4')  # Sincle colab has no option for live recording, existing file is used

In [52]:
def detect_face_video(img):
    label_dict={0:'Female',1:'Male'}
    color_dict={0:(0,255,0),1:(0,0,255)}
    
    face_coord = face_cascade.detectMultiScale(img,scaleFactor = 1.2,minNeighbors = 5)  # This gives us the coordinates of rectangle drawn across the face 
    for x,y,w,h in face_coord:
        image = img[y:y+h,x:x+h]
        image_gry = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        resize_img = cv2.resize(image_gry,(200,200))
        norm_img = resize_img/255.0
        reshaped=np.reshape(norm_img,(1,200,200,1))
        #reshaped = np.vstack([reshaped])

        try:  
            with tf.device('/device:GPU:5'):
                gender_res = gen_model.predict(reshaped)
        except RuntimeError as e:
            print(e)
        if(gender_res[0][0]<0.5):
            label=0
        else:
            label=1
     
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],thickness = 5)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, label_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    return img

In [53]:
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('test3_out.mp4',cv2.VideoWriter_fourcc(*'XVID'),25,(width,height))

* colab gives the output of video files as a series of each frame present in video file

In [54]:
try:  
    with tf.device('/device:GPU:5'):
        gen_model = load_model("models/cnn1_relu_sgd_84acc_80tr_20te_less_false_neg")
except RuntimeError as e:
    print(e)

In [55]:
if video.isOpened() == False:
  print('Error! File Not found')
while video.isOpened():
    ret,frame = video.read()
    frame = detect_face_video(frame)  #detect the face for each frame displayed on video
    if ret == True:
        out.write(frame)
#     if cv2.waitKey(27) & 0xFF == ord('q'): #if live webcam is given input then video can be quit using letter 'q'
#         break
    else:
        break
video.release()
out.release()
cv2.destroyAllWindows()